In [1]:
import numpy as np
import glob, os
from keras import layers as ly
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import RepeatVector
from keras.layers import TimeDistributed
from keras.preprocessing import sequence
from sklearn.model_selection import train_test_split
import keras.backend.tensorflow_backend as K
from keras.optimizers import Adam
from keras.layers import Input, LSTM, RepeatVector, Lambda
from keras.models import Model

Using TensorFlow backend.


In [2]:
dir = './new_sequence_dfs/*'

# file read
all_data = []
sequence_length = []
name = []
for file in sorted(glob.glob(dir)):
    name.append(file.split('/')[2].replace('.txt', ''))
    datasets = []
    for f in open(file, 'r'):
        f = f.replace(']', '').replace('[', '').replace('\n','')
        (u, v, w) = f.split(',')
        datasets.append([int(u), int(v), float(w)])
    sequence_length.append(len(datasets))
    all_data.append(datasets)
#all_data = np.array(all_data)
all_data = np.array([np.array(arr) for arr in all_data])

In [3]:
x_train, x_test, train_name, test_name = train_test_split(all_data, name, test_size=0.3)
x_test, x_val, test_name, val_name = train_test_split(x_test, test_name, test_size=0.33)

In [4]:
n_features = 3
#batch_size = 32
epochs = 500
steps_per_epoch = len(x_train)

In [5]:
def repeat_vector(args):
    layer_to_repeat = args[0]
    sequence_layer = args[1]
    return RepeatVector(K.shape(sequence_layer)[1])(layer_to_repeat)

In [6]:
inputs = Input(shape=(None, n_features))
encoded = LSTM(128, return_sequences=True)(inputs)  #activation 안적으면 tanh
encoded = LSTM(64, return_sequences=True)(encoded)
encoded = LSTM(32, activation='relu')(encoded)

decoded = Lambda(repeat_vector, output_shape=(None, 32)) ([encoded, inputs]) # inputs의 shape[1] 만큼 encoded 를 반복 생성

decoded = LSTM(32, return_sequences=True)(decoded)
decoded = LSTM(64, return_sequences=True)(decoded)
decoded = LSTM(128, return_sequences=True)(decoded)
decoded = TimeDistributed(Dense(n_features))(decoded)
encoder = Model(inputs, encoded)

W0919 15:03:56.744072 140428727514880 deprecation_wrapper.py:119] From /home/minji/.local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:66: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0919 15:03:56.752596 140428727514880 deprecation_wrapper.py:119] From /home/minji/.local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:541: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0919 15:03:56.754572 140428727514880 deprecation_wrapper.py:119] From /home/minji/.local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4432: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.



In [7]:
lstm_autoencoder = Model(inputs, decoded)
lstm_autoencoder.compile(loss='mean_squared_error', optimizer=Adam(lr=0.001))

W0919 15:03:57.725203 140428727514880 deprecation_wrapper.py:119] From /home/minji/.local/lib/python3.7/site-packages/keras/optimizers.py:793: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.



In [8]:
def train_generator(x_train):
    idx = 0
    while True:
        yield np.array([x_train[idx]]), np.array([x_train[idx]])
        idx +=1
        if idx >= len(x_train):
            idx = 0

In [9]:
lstm_autoencoder.fit_generator(train_generator(x_train), epochs=epochs, steps_per_epoch=steps_per_epoch, verbose=1)#, validation_data=val_generator(x_val))


W0919 15:03:57.991599 140428727514880 deprecation.py:323] From /home/minji/anaconda3/envs/graph/lib/python3.7/site-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W0919 15:04:00.151787 140428727514880 deprecation_wrapper.py:119] From /home/minji/.local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:1033: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.

W0919 15:04:00.328361 140428727514880 deprecation_wrapper.py:119] From /home/minji/.local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:1020: The name tf.assign is deprecated. Please use tf.compat.v1.assign instead.



Epoch 1/500
441/441 [==============================] - 10s 22ms/step - loss: 36.0578
Epoch 2/500
441/441 [==============================] - 7s 15ms/step - loss: 23.9145
Epoch 3/500
441/441 [==============================] - 7s 16ms/step - loss: 21.4597
Epoch 4/500
441/441 [==============================] - 7s 15ms/step - loss: 19.8140
Epoch 5/500
441/441 [==============================] - 7s 15ms/step - loss: 18.8939
Epoch 6/500
441/441 [==============================] - 7s 15ms/step - loss: 15.7749
Epoch 7/500
441/441 [==============================] - 7s 15ms/step - loss: 13.5090
Epoch 8/500
441/441 [==============================] - 7s 15ms/step - loss: 12.7391
Epoch 9/500
441/441 [==============================] - 7s 15ms/step - loss: 10.2392
Epoch 10/500
441/441 [==============================] - 7s 15ms/step - loss: 11.5081
Epoch 11/500
441/441 [==============================] - 7s 15ms/step - loss: 9.5215
Epoch 12/500
441/441 [==============================] - 7s 15ms/step - los

441/441 [==============================] - 7s 15ms/step - loss: 4.3633
Epoch 99/500
441/441 [==============================] - 7s 15ms/step - loss: 3.4335
Epoch 100/500
441/441 [==============================] - 7s 15ms/step - loss: 0.6170
Epoch 101/500
441/441 [==============================] - 7s 17ms/step - loss: 0.3581
Epoch 102/500
441/441 [==============================] - 7s 16ms/step - loss: 0.3527
Epoch 103/500
441/441 [==============================] - 7s 16ms/step - loss: 0.2936
Epoch 104/500
441/441 [==============================] - 7s 17ms/step - loss: 0.2788
Epoch 105/500
441/441 [==============================] - 7s 16ms/step - loss: 0.3278
Epoch 106/500
441/441 [==============================] - 7s 17ms/step - loss: 0.8861
Epoch 107/500
441/441 [==============================] - 7s 16ms/step - loss: 4.3339
Epoch 108/500
441/441 [==============================] - 7s 16ms/step - loss: 1.2866
Epoch 109/500
441/441 [==============================] - 7s 15ms/step - loss: 0.

441/441 [==============================] - 7s 15ms/step - loss: 6.0185
Epoch 195/500
441/441 [==============================] - 7s 15ms/step - loss: 1.8133
Epoch 196/500
441/441 [==============================] - 7s 15ms/step - loss: 1.2458
Epoch 197/500
441/441 [==============================] - 7s 15ms/step - loss: 0.3145
Epoch 198/500
441/441 [==============================] - 7s 15ms/step - loss: 0.2164
Epoch 199/500
441/441 [==============================] - 7s 15ms/step - loss: 0.1654
Epoch 200/500
441/441 [==============================] - 7s 15ms/step - loss: 0.1356
Epoch 201/500
441/441 [==============================] - 7s 15ms/step - loss: 0.1217
Epoch 202/500
441/441 [==============================] - 7s 15ms/step - loss: 0.9749
Epoch 203/500
441/441 [==============================] - 7s 15ms/step - loss: 0.7654
Epoch 204/500
441/441 [==============================] - 7s 15ms/step - loss: 0.1533
Epoch 205/500
441/441 [==============================] - 7s 15ms/step - loss: 0

441/441 [==============================] - 5s 11ms/step - loss: 0.0621
Epoch 291/500
441/441 [==============================] - 5s 11ms/step - loss: 0.0689
Epoch 292/500
441/441 [==============================] - 5s 11ms/step - loss: 0.1955
Epoch 293/500
441/441 [==============================] - 5s 11ms/step - loss: 0.3050
Epoch 294/500
441/441 [==============================] - 5s 11ms/step - loss: 0.1304
Epoch 295/500
441/441 [==============================] - 5s 11ms/step - loss: 0.0924
Epoch 296/500
441/441 [==============================] - 5s 11ms/step - loss: 0.0638
Epoch 297/500
441/441 [==============================] - 5s 11ms/step - loss: 0.4388
Epoch 298/500
441/441 [==============================] - 5s 11ms/step - loss: 2.1878
Epoch 299/500
441/441 [==============================] - 5s 11ms/step - loss: 0.1827
Epoch 300/500
441/441 [==============================] - 5s 11ms/step - loss: 0.0866
Epoch 301/500
441/441 [==============================] - 5s 12ms/step - loss: 0

441/441 [==============================] - 5s 11ms/step - loss: 0.0470
Epoch 387/500
441/441 [==============================] - 5s 11ms/step - loss: 0.0575
Epoch 388/500
441/441 [==============================] - 5s 11ms/step - loss: 0.0867
Epoch 389/500
441/441 [==============================] - 5s 11ms/step - loss: 0.0812
Epoch 390/500
441/441 [==============================] - 5s 11ms/step - loss: 0.0679
Epoch 391/500
441/441 [==============================] - 5s 11ms/step - loss: 0.0408
Epoch 392/500
441/441 [==============================] - 5s 11ms/step - loss: 0.0419
Epoch 393/500
441/441 [==============================] - 5s 12ms/step - loss: 0.3282
Epoch 394/500
441/441 [==============================] - 5s 11ms/step - loss: 1.7112
Epoch 395/500
441/441 [==============================] - 5s 11ms/step - loss: 0.7356
Epoch 396/500
441/441 [==============================] - 5s 11ms/step - loss: 0.1382
Epoch 397/500
441/441 [==============================] - 5s 11ms/step - loss: 0

441/441 [==============================] - 5s 12ms/step - loss: 0.0342
Epoch 483/500
441/441 [==============================] - 5s 12ms/step - loss: 0.0783
Epoch 484/500
441/441 [==============================] - 5s 12ms/step - loss: 0.4859
Epoch 485/500
441/441 [==============================] - 5s 11ms/step - loss: 1.2145
Epoch 486/500
441/441 [==============================] - 5s 11ms/step - loss: 0.1914
Epoch 487/500
441/441 [==============================] - 5s 11ms/step - loss: 0.0762
Epoch 488/500
441/441 [==============================] - 5s 11ms/step - loss: 0.0479
Epoch 489/500
441/441 [==============================] - 5s 11ms/step - loss: 0.0338
Epoch 490/500
441/441 [==============================] - 5s 11ms/step - loss: 0.0281
Epoch 491/500
441/441 [==============================] - 5s 11ms/step - loss: 0.0225
Epoch 492/500
441/441 [==============================] - 5s 11ms/step - loss: 0.0185
Epoch 493/500
441/441 [==============================] - 5s 11ms/step - loss: 0

In [10]:
from keras.models import load_model
# model save
lstm_autoencoder.save('lstm_autoencoder_new_dfs.h5')

In [11]:
loaded_model = load_model('lstm_autoencoder_new_dfs.h5')